In [1]:
import torch
import torch.nn.functional as F
import numpy as np                 # import numpy
import matplotlib.pyplot as plt    # import matplotlib
from torch.utils.data import Dataset, DataLoader,random_split
import torch.optim as optim
from tqdm import tqdm
import h5py
import scipy
from scipy.signal import find_peaks
from IPython.display import Image
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import optuna

import utils
import Model
import Simulate_Dataset

# Set a random seed for PyTorch
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [2]:
# dataset parameters
fs = 1000                     # sampling frequency in Hz
data_dur =  3*63                 # data duration in s
actpot_dur = 5                   # action potential length in ms
window_dur = 3                   # window dutation in s
C = 1                              # number of neurons
s = 40                              # number of appearance of each neuron in each window
x_mean = 2                         # code amplitude mean
x_std = 0.1                         # code amplitude std
noise_std = 0.1
snr = 20

K = int(actpot_dur * fs / 1000) # filter legnth
N = int(window_dur * fs)        # example duration
J = int(data_dur / window_dur)  # number of examples

# data parameters
data_hyp ={"J": J, "N": N, "K": K, "C": C, "s": s, "x_mean": x_mean,
           "x_std": x_std, "SNR": snr, "device": device}

print("filter length:", K)
print("example duration:", N)
print("number of examples:", J)

filter length: 5
example duration: 3000
number of examples: 63


In [3]:
# fill in the gaps
net_hyp = {"T": 200,                 # number of encoder unfolding
           "L": 10,                  # 1/L is the step size in the encoder
           "C": 1,                 # number of filters
           "K": 18,                  # legnth of the filters
           "lam": 0.2,
           "device": device
           }

net = Model.CRsAE1D(net_hyp)

# training parameters
train_hyp = {"batch_size": 8, "num_epochs": 12, "lr": 1e-2, "shuffle": True}
# criterion
criterion = torch.nn.MSELoss()
# optimizer
optimizer = optim.Adam(net.parameters(), lr=train_hyp["lr"], eps=1e-3)

In [4]:
snr_list = np.linspace(-10,50,13)
print(snr_list)

[-10.  -5.   0.   5.  10.  15.  20.  25.  30.  35.  40.  45.  50.]


In [13]:
def f1_score(label, pred):
  label = np.array(label.cpu())
  true_positives = np.sum((pred == 1) & (label == 1))
  all_positives = np.sum(label == 1)
  predicted_positives = np.sum(pred == 1)

  precision = true_positives / predicted_positives if predicted_positives > 0 else 0.0
  recall = true_positives / all_positives if all_positives > 0 else 0.0

  f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
  return f1_score

def check_naive_classifiers(signal, label, k=10):
    dataset_behave = create_behave_dataset_matrix(signal,label,device = device)
    X = dataset_behave.data.cpu()
    y = dataset_behave.label.cpu()

    kf = KFold(n_splits=k, shuffle=False)
    test_accuracy_list = []
    print("running tuna study on metric")
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=False)

        def objective(trial):
            # Define hyperparameter search space
            model_type = trial.suggest_categorical('model', ['svm_linear', 'logistic'])
            C = trial.suggest_float('C', 0.01, 100, log=True)

            if model_type.startswith('svm'):
                svm_classifier = svm.SVC(C=C, kernel='linear')
                svm_classifier.fit(X_train, y_train)
                y_val_pred = svm_classifier.predict(X_val)
                val_accuracy = f1_score(y_val, y_val_pred)
            else:
                logistic_classifier = LogisticRegression(C=C, max_iter=1000)
                logistic_classifier.fit(X_train, y_train)
                y_val_pred = logistic_classifier.predict(X_val)
                val_accuracy = f1_score(y_val, y_val_pred)

            return val_accuracy

        # Create an Optuna study
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)

        # Get the best trial
        best_trial = study.best_trial
        best_params = best_trial.params
        best_score = best_trial.value

        # Train the best model on the entire training set
        if best_params['model'].startswith('svm'):
            best_model = svm.SVC(C=best_params['C'], kernel=best_params['model'][4:])
        else:
            best_model = LogisticRegression(C=best_params['C'], max_iter=1000)
        best_model.fit(X_train, y_train)

        # Evaluate the model on the test set
        y_test_pred = best_model.predict(X_test)
        test_accuracy = f1_score(y_test, y_test_pred)

        print("Best Hyperparameters from Optuna:", best_params)
        print("Best Validation Set Accuracy:", best_score)
        print("Test Set Accuracy:", test_accuracy)
        test_accuracy_list.append(test_accuracy)

    return np.mean(test_accuracy_list)


class create_behave_dataset_matrix(Dataset):
    def __init__(self, x,label,device):
        self.label = label
        chunks = torch.split(x, 1, dim=1)
        # Concatenating the chunks along the first dimension
        self.data = x.T

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        return  self.data[idx], self.label[idx]

In [14]:
snr_list = [(10,0.15),(15,0.15),(20,0.1),(25,0.05),(30,0.05),(35,0.1),(40,0),(45,0),(50,0),(-10,0.9),(-5,0.5),(0,0.3),(5,0.25)]
#snr_list = [(50,0)]

In [15]:
x_f1_score = []
y_f1_score = []
y_hat_f1_score = []
cur_snr_list = []
x_const = None

In [ ]:
for itr in range(20):
  print(f"%%%%%%%%%%%%%%{itr}%%%%%%%%%%%%%%%%%")
  for cur_snr, cur_lam in snr_list:
    net_hyp = {"T": 200,                 # number of encoder unfolding
            "L": 10,                  # 1/L is the step size in the encoder
            "C": 2,                 # number of filters
            "K": 18,                  # legnth of the filters
            "lam": cur_lam,
            "device": device
            }
    data_hyp ={"J": J, "N": N, "K": K, "C": C, "s": s, "x_mean": x_mean,
            "x_std": x_std, "SNR": cur_snr, "device": device}
    dataset = Simulate_Dataset.SimulatedDataset1D(data_hyp,x_const,one_kernel = True)
    net = Model.CRsAE1D(net_hyp)
    net.load_state_dict(torch.load(f'pretrained_on_simulated_data\snr_{snr_list[0][0]}.pth'))
    with torch.no_grad():
      y,x,_,_ = dataset[:]
      x_const = x
      x = torch.sum(x, dim=1, keepdim=True)
      y_hat, x_hat = net(y)
      #utils.vis_data_est(y[1,0], y_hat[1,0])
      x_hat = torch.sum(x_hat, dim=1, keepdim=True)
      y = y.squeeze(dim = 1)
      b = x[0,0] > 0
      x = x.squeeze(dim = 1)
      y_hat = y_hat.squeeze(dim = 1)
      x_hat = x_hat.squeeze(dim = 1)
      padding_begining = 7
      padding_end = 10
      # Perform the padding
      padded_tensor = torch.cat([torch.zeros(padding_begining).to(device), torch.tensor(b),torch.zeros(padding_end).to(device)])
      modified_tensor = torch.zeros_like(padded_tensor).to(device)
      indices_of_ones = torch.nonzero(padded_tensor).squeeze()

      for index in indices_of_ones:
        modified_tensor[index-1:index+3] = 1



    x_accuracy = []
    y_accuracy = []
    y_hat_accuracy = []
    print(f"For SNR = {cur_snr}")
    print("On x")
    x_accuracy.append(check_naive_classifiers(x, b))
    print("On y")
    y_accuracy.append(check_naive_classifiers(y, modified_tensor))
    print("On y_hat")
    y_hat_accuracy.append(check_naive_classifiers(y_hat, modified_tensor))

    cur_snr_list.append(cur_snr)
    x_f1_score.append(max(x_accuracy))
    y_f1_score.append(max(y_accuracy))
    y_hat_f1_score.append(max(y_hat_accuracy))

  # reset x for new study
  x_const = None

%%%%%%%%%%%%%%0%%%%%%%%%%%%%%%%%


C:\Users\yonatan.kle\AppData\Local\Temp\ipykernel_13436\3175999884.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_tensor = torch.cat([torch.zeros(padding_begining).to(device), torch.tensor(b),torch.zeros(padding_end).to(device)])
[I 2023-09-12 13:47:00,654] A new study created in memory with name: no-name-45037bf9-b7a2-463b-9ab7-0b5e23608bb3
[I 2023-09-12 13:47:00,670] Trial 0 finished with value: 1.0 and parameters: {'model': 'logistic', 'C': 2.9363620902474685}. Best is trial 0 with value: 1.0.
[I 2023-09-12 13:47:00,678] Trial 1 finished with value: 1.0 and parameters: {'model': 'svm_linear', 'C': 0.07864651250639575}. Best is trial 0 with value: 1.0.
[I 2023-09-12 13:47:00,683] Trial 2 finished with value: 1.0 and parameters: {'model': 'svm_linear', 'C': 27.471085678865293}. Best is trial 0 with value: 1.0.
[I 2023-09-1

For SNR = 10
On x
running tuna study on metric


[I 2023-09-12 13:47:00,853] Trial 16 finished with value: 1.0 and parameters: {'model': 'logistic', 'C': 2.1124446812597553}. Best is trial 0 with value: 1.0.
[I 2023-09-12 13:47:00,862] Trial 17 finished with value: 1.0 and parameters: {'model': 'svm_linear', 'C': 10.280725548389247}. Best is trial 0 with value: 1.0.
[I 2023-09-12 13:47:00,879] Trial 18 finished with value: 1.0 and parameters: {'model': 'logistic', 'C': 0.3325567752251213}. Best is trial 0 with value: 1.0.
[I 2023-09-12 13:47:00,888] Trial 19 finished with value: 1.0 and parameters: {'model': 'svm_linear', 'C': 0.9899561911748959}. Best is trial 0 with value: 1.0.
[I 2023-09-12 13:47:00,905] A new study created in memory with name: no-name-d00f7ac8-a690-49a3-9f42-7b6cd4dea104
[I 2023-09-12 13:47:00,912] Trial 0 finished with value: 1.0 and parameters: {'model': 'svm_linear', 'C': 0.20413116226584274}. Best is trial 0 with value: 1.0.
[I 2023-09-12 13:47:00,921] Trial 1 finished with value: 1.0 and parameters: {'model'

Best Hyperparameters from Optuna: {'model': 'logistic', 'C': 2.9363620902474685}
Best Validation Set Accuracy: 1.0
Test Set Accuracy: 1.0


...

[I 2023-09-12 13:48:41,823] Trial 18 finished with value: 0.6666666666666666 and parameters: {'model': 'logistic', 'C': 0.08774640181029439}. Best is trial 0 with value: 1.0.
[I 2023-09-12 13:48:41,825] Trial 19 finished with value: 1.0 and parameters: {'model': 'svm_linear', 'C': 0.023620442513599824}. Best is trial 0 with value: 1.0.
[I 2023-09-12 13:48:41,844] A new study created in memory with name: no-name-71b7bb8b-3613-4776-be7e-409adf0c1ab7
[I 2023-09-12 13:48:41,859] Trial 0 finished with value: 0.0 and parameters: {'model': 'logistic', 'C': 0.4772620990589344}. Best is trial 0 with value: 0.0.
[I 2023-09-12 13:48:41,859] Trial 1 finished with value: 0.0 and parameters: {'model': 'logistic', 'C': 0.4025522545499814}. Best is trial 0 with value: 0.0.
[I 2023-09-12 13:48:41,878] Trial 2 finished with value: 0.0 and parameters: {'model': 'logistic', 'C': 0.7760112913482392}. Best is trial 0 with value: 0.0.


Best Hyperparameters from Optuna: {'model': 'svm_linear', 'C': 0.07174259905017222}
Best Validation Set Accuracy: 1.0
Test Set Accuracy: 1.0
On y
running tuna study on metric


[I 2023-09-12 13:48:43,272] Trial 3 finished with value: 0.0 and parameters: {'model': 'svm_linear', 'C': 47.71033331714876}. Best is trial 0 with value: 0.0.
[I 2023-09-12 13:48:45,272] Trial 4 finished with value: 0.0 and parameters: {'model': 'svm_linear', 'C': 69.32340499262675}. Best is trial 0 with value: 0.0.
[I 2023-09-12 13:48:45,310] Trial 5 finished with value: 0.0 and parameters: {'model': 'svm_linear', 'C': 0.027039512744579552}. Best is trial 0 with value: 0.0.
[I 2023-09-12 13:48:45,318] Trial 6 finished with value: 0.0 and parameters: {'model': 'logistic', 'C': 0.9866844485561863}. Best is trial 0 with value: 0.0.
[I 2023-09-12 13:48:45,376] Trial 7 finished with value: 0.0 and parameters: {'model': 'svm_linear', 'C': 0.41482656093686976}. Best is trial 0 with value: 0.0.
[I 2023-09-12 13:48:45,383] Trial 8 finished with value: 0.0 and parameters: {'model': 'logistic', 'C': 0.30677225619194487}. Best is trial 0 with value: 0.0.
[I 2023-09-12 13:48:45,384] Trial 9 finish

Best Hyperparameters from Optuna: {'model': 'logistic', 'C': 0.4772620990589344}
Best Validation Set Accuracy: 0.0
Test Set Accuracy: 0.0


[I 2023-09-12 13:48:45,909] Trial 6 finished with value: 0.0 and parameters: {'model': 'svm_linear', 'C': 7.184910556186215}. Best is trial 0 with value: 0.0.
[I 2023-09-12 13:48:45,925] Trial 7 finished with value: 0.0 and parameters: {'model': 'logistic', 'C': 0.4924871421034774}. Best is trial 0 with value: 0.0.
[I 2023-09-12 13:48:45,928] Trial 8 finished with value: 0.0 and parameters: {'model': 'logistic', 'C': 0.2554563771589932}. Best is trial 0 with value: 0.0.
[I 2023-09-12 13:48:45,944] Trial 9 finished with value: 0.0 and parameters: {'model': 'logistic', 'C': 83.96665157153275}. Best is trial 0 with value: 0.0.
[I 2023-09-12 13:48:46,522] Trial 10 finished with value: 0.0 and parameters: {'model': 'svm_linear', 'C': 16.368848012265392}. Best is trial 0 with value: 0.0.


In [ ]:
yi,xi,h,_ = dataset[0:3]
yi_hat, xi_hat = net(yi)

utils.vis_data_est(yi[0,0][0:500], modified_tensor[0:500])
utils.vis_data_est(yi[0,0][0:500], yi_hat[0,0][0:500])


print(len(padded_tensor))
print(len(yi[0,0]))
utils.vis_data(padded_tensor[0:500])
#utils.vis_data(b)
utils.vis_filters(h)

In [ ]:
plt.scatter(cur_snr_list,x_f1_score, marker='o')
plt.xlabel('SNR')
plt.ylabel('x F1 score')
plt.title('')
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(cur_snr_list,y_f1_score, marker='o')
plt.xlabel('SNR')
plt.ylabel('y F1 score')
plt.title('')
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(cur_snr_list, y_hat_f1_score, marker='o')
plt.scatter(cur_snr_list,y_f1_score, marker='*')
plt.xlabel('SNR')
plt.ylabel('F1 score')
plt.legend(['y_hat','y'])
plt.title('F1 score before and after denoising')
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
data = {'SNR': cur_snr_list, 'F1 score y_hat': y_hat_f1_score, 'F1 score y': y_f1_score}
df = pd.DataFrame(data)

# Specify the file name
csv_filename = 'simulated_behave_data_comperison.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_filename, index=False)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.pandas.read_csv('simulated_behave_data_comperison.csv')

# Group the data by SNR and calculate mean and variance
grouped = df.groupby('SNR').agg({'F1 score y_hat': ['mean', 'std','count']})

# Extract the mean and variance values
means = grouped['F1 score y_hat']['mean']
std = grouped['F1 score y_hat']['std']
standard_error = std/grouped['F1 score y_hat']['count']

# Create a bar plot with error bars
plt.errorbar(means.index, means, yerr=standard_error, fmt='o', capsize=4)
# Group the data by SNR and calculate mean and variance
grouped = df.groupby('SNR').agg({'F1 score y': ['mean', 'std','count']})

# Extract the mean and variance values
means = grouped['F1 score y']['mean']
std = grouped['F1 score y']['std']
standard_error = std/grouped['F1 score y']['count']

# Create a bar plot with error bars
plt.errorbar(means.index, means, yerr=standard_error, fmt='o', capsize=4)

# Add labels and title
plt.xlabel('SNR')
plt.ylabel('Mean Values')
plt.title('Mean Values with standard Error Bars')
plt.legend(['y_hat','y'])

# Show the plot
plt.show()